In [1]:
import torch
import numpy as np
import os
import nltk
import torch.nn as nn
import torchvision.models as models
import warnings
warnings.filterwarnings('ignore') # make notebook more visible, disable warnings

from constants import config_data
from dataset_factory import get_datasets
from pycocotools.coco import COCO
from matplotlib import pyplot as plt
from model_factory import baseline_encoder, baseline_decoder
from torchvision import transforms

torch.manual_seed(128)
np.random.seed(seed = 100) # fix the random seed to make things reproducible

## Setup GPU(cuda)

In [2]:
#Setup GPU settings
print("GPU is availiable:", torch.cuda.is_available())
print("# of GPU:", torch.cuda.device_count())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Can run on GPU") 
else:
    device = torch.device("cpu")
    print("can only run on CPU")
    
print("Run on:", device)

GPU is availiable: True
# of GPU: 1
Can run on GPU
Run on: cuda:0


## Load the configration, and possibily adjust some of them

In [3]:
config_data['dataset']['batch_size'] = 20
config_data['dataset']['num_workers'] = 4

config_data

{'dataset': {'images_root_dir': 'data/images',
  'training_ids_file_path': 'data/train_ids.csv',
  'validation_ids_file_path': 'data/val_ids.csv',
  'test_ids_file_path': 'data/test_ids.csv',
  'training_annotation_file_path': 'data/annotations/captions_train2014.json',
  'test_annotation_file_path': 'data/annotations/captions_val2014.json',
  'vocabulary_threshold': 10,
  'img_size': (256, 256),
  'batch_size': 20,
  'num_workers': 4},
 'model': {'hidden_size': 512,
  'embedding_size': 300,
  'model_type': 'baseline'}}

In [4]:
train_augmentation = transforms.Compose([
    transforms.RandomResizedCrop(size=(256, 256),scale=(0.08, 1.0),ratio=(0.75, 1.3333)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=[-45.0, 45.0]),
    transforms.RandomAffine(degrees=[-45.0, 45.0]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Get dataloaders

In [5]:
# get dataloaders 
coco_test, vocabulary, train_data_loader, val_data_loader, test_data_loader = get_datasets(config_data, transform=None)

loading annotations into memory...
Done (t=0.62s)
creating index...
index created!
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
Using the saved vocab.
loading annotations into memory...
Done (t=0.60s)
creating index...
index created!
loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


# Train

#### lowest validation loss 1.46
#### embed_size=512, hidden_size=600

## Prepare model

In [6]:
vocab_size = len(vocabulary)
embed_size = config_data['model']['embedding_size']
hidden_size = config_data['model']['hidden_size']

embed_size = 500 # 300
hidden_size = 1000 # 512

print("vocab_size:", vocab_size) # see the vocab size for convininence to check dimensionalities
print("embed_size:", embed_size)
print("hidden_size:", hidden_size)

vocab_size: 1945
embed_size: 500
hidden_size: 1000


In [7]:
# get the model
encoder = baseline_encoder(embed_size).to(device)
decoder = baseline_decoder(embed_size, hidden_size, vocab_size, device).to(device)

# get training ready
criterion = nn.CrossEntropyLoss()
params = list(decoder.parameters())+list(encoder.resnet.fc.parameters())
optimizer = torch.optim.Adam(params, lr=0.00005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# specify the name of checkpoints
cp_name_encoder = "LSTM_encoder.pt"
cp_name_decoder = "LSTM_decoder.pt"

## Train and validation

In [ ]:
train_loss = [] # used to record training loss per epoch
val_loss = [] # used to record validation loss per epoch
best_val_loss = float('inf') # used to record the best validation loss for saving the best model

for epoch in range(0, 20):
    # turn the train mode on
    encoder.train()
    decoder.train()
    
    # used to record losses per batch
    # average them as the loss per epoch after the epoch
    train_l_avg = [] 
    val_l_avg = []
    
    # go through the train set
    for batch_idx, (image, target, img_ids) in enumerate(train_data_loader):
        image, target = image.to(device), target.to(device) # set to GPU
        
        # zero gradients
        decoder.zero_grad()
        encoder.zero_grad()
        
        # get the image embedding
        feature = encoder(image)
        # pass the caption(target) and image embedding to the decoder
        output = decoder(feature, target)
        
        # gradient descendent
        loss = criterion(output.view(-1, vocab_size), target.view(-1))
        loss.backward()
        optimizer.step()
        
        train_l_avg.append(loss.item()) # record loss per batch
        
    # average loss per batch to get loss per epoch
    train_l = np.mean(train_l_avg) 
    train_loss.append(train_l)
    
    # turn the evaluation mode on
    encoder.eval()
    decoder.eval()
    
    with torch.no_grad(): 
        # go through the validation set
        for batch_idx, (image, target, img_ids) in enumerate(val_data_loader):
            image, target = image.to(device), target.to(device) # set to GPU
            
            feature = encoder(image) # get the image embedding
            output = decoder(feature, target) # pass the caption(target) and image embedding to the decoder
            
            # get validation loss per batch
            loss = criterion(output.view(-1, vocab_size), target.view(-1))
            val_l_avg.append(loss.item())
            
    # average loss per batch to get loss per epoch
    val_l = np.mean(val_l_avg)
    val_loss.append(val_l)
    
    # if the validation loss is less, we have a better model
    if val_l < best_val_loss:
        best_val_loss = val_l
        torch.save(encoder.state_dict(), cp_name_encoder)
        torch.save(decoder.state_dict(), cp_name_decoder)
        
    #scheduler.step()
            
    print("Epoch", epoch + 1, "done:")
    print("training loss:", train_l)
    print("validation loss:", val_l, "\n")

## Train and validation loss plot

In [ ]:
plt.plot(train_loss)
plt.plot(val_loss)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train/Validation Loss")
plt.legend(["Training Loss", "Validation Loss"])

# Evaluate by Test

In [ ]:
from caption_utils import bleu1, bleu4 # provided functions to calculate scores

## Load the model

In [ ]:
encoder = baseline_encoder(embed_size)
decoder = baseline_decoder(embed_size, hidden_size, vocab_size, device)

encoder.load_state_dict(torch.load(cp_name_encoder))
decoder.load_state_dict(torch.load(cp_name_decoder))

encoder = encoder.to(device)
decoder = decoder.to(device)

## Some functions helps us to evaluate BLEU scores

In [ ]:
def caption2sentence(captions, vocabulary):
    """takes captions(list of word_idx) predicted by the model, and convert the captions to sentence(list of words)"""
    # get a batch of captions (word idx)
    
    out = []
    
    for i in range(len(captions)): # iterate through batch
        word_list = []
        
        for word_idx in captions[i]:
            word_list.append(vocabulary.idx2word[word_idx]) 
            
            if word_idx == 2: # if the word_idx == 2, <end>
                break
            
        word_list = word_list[1:-1] # discard <start> and <end>
        out.append(word_list)
        
    return out

In [ ]:
def get_reference_caption(img_id, coco_test):
    """takes the img_id to get all captions(list of word_idx) of the image as reference to calcualte BLEU scores"""
    out = []
    for data in coco_test.imgToAnns[img_id]: # for each caption of given img_id
        # tokenize the captions to make it compatible with the BLEU scores
        caption_token = []
        ref_cap = data["caption"] 
        caption_tokenized = nltk.tokenize.word_tokenize(ref_cap.lower())
        out.append(caption_tokenized)
        
    return out

## Evaluate

#### baseline: 1.49/67 BLEU1
#### lowest loss: 1.51

In [ ]:
# turn the evaluation mode on
encoder.eval()
decoder.eval()

test_l_avg = [] # used to calculate test loss, stores loss per batch
score1_avg = [] # used to calculate BLEU1, stores BLEU1 per batch
score4_avg = [] # used to calculate BLEU4, stores BLEU4 per batch

with torch.no_grad():
    # go through the test set
    for batch_idx, (image, target, img_ids) in enumerate(test_data_loader):
        image, target = image.to(device), target.to(device) # set to GPU
            
        # forward the model
        feature = encoder(image)
        output = decoder(feature, target)
            
        # get loss per batch
        loss = criterion(output.view(-1, vocab_size), target.view(-1))
        test_l_avg.append(loss.item())
        
        # predict according to the image embedding
        feat = feature.unsqueeze(1) # reshape to make it can be predicted by the model
        predicted_captions = decoder.predict(feat)
        # convert predictions(list of word_idx) into sentence(list of words)
        predicted_captions = caption2sentence(predicted_captions, vocabulary)
        
        for i in range(len(predicted_captions)): # go through each element in the batch
            predicted = predicted_captions[i]
            im_id = img_ids[i] # get the img_id of the image
            reference = get_reference_caption(im_id, coco_test) # get reference for the image
            
            # get scores
            score1 = bleu1(reference, predicted)
            score4 = bleu4(reference, predicted)
            
            # print some examples to make sure everything goes alright
            if i == 0 and batch_idx== 0:
                print("predicted", predicted)
                print("reference", reference)
                print("bleu1:", score1)
                print("bleu4:", score4, "\n")
        
            # record scores per element
            score1_avg.append(score1)
            score4_avg.append(score4)
        
print("Test loss is:", np.mean(test_l_avg)) # test loss by averaging loss per batch
# scores get by averaging scores of all images
print("BLEU1 is:", np.mean(score1_avg))
print("BLEU4 is:", np.mean(score4_avg))